# Exploratory Data Analysis
spiegare quali siano gli obbiettivi per i quali stiamo preparando i dataset



## Import Packages
* Il pacchetto **numpy** serve a lavorare con valori e array numerici. Performa efficienti conti matematici e statistici.
* Il pacchetto **pandas** serve a manipolare, pulire e gestire dati strutturati attraverso strutture come i DataFrame.
* Il pacchetto **geopandas** è un'estensione del pacchetto *pandas* e serve per gestire dati geospaziali e visualizzarli in mappe.

In [65]:
import pandas as pd
import numpy as np
import geopandas as gp

## Importa Data

Il primo DataFrame contiene i dati sui consumi delle linee elettriche di tutto il Trentino.  
Per ogni linea elettrica viene fornito il dato di consumo ogni 10 minuti per l'intero mese.  
I dati sono per il mese di Novembre 2013 (il processo verrà ripetuto per il mese di Dicembre 2013).


Il secondo DataFrame contiene i dati riguardanti le ubicazioni e linee elettriche presenti in ogni zona del Trentino.  
Per ogni zona del Trentino (identificata da *SQUAREID*) vengono associate le linee elettriche da lì passanti (*LINESET*) e per ogni combinazione di zona e linea elettrica viene associato il numero di ubicazioni presenti (*NR_UBI*)



In [123]:
file_path_proc = "./data/processed/"
file_path_raw = "./data/raw/"
set_df = pd.read_csv(file_path_proc + "SET-nov-2013.csv")
lin_df = pd.read_csv(file_path_proc + "line.csv")
set_df[:5]

,LINESET,POWER,DAY,MINUTE,WEEK_DAY
0,DG1000420,32348.159136,1,0,4
1,DG1000420,32348.159136,1,10,4
2,DG1000420,31104.000000,1,20,4
3,DG1000420,30481.919136,1,30,4
4,DG1000420,30481.919136,1,40,4


## Trasformazione colonna *DATE*
Trasformiamo la colonna *DATE* in tre colonne separate che indicano il giorno (*DAY*), i minuti (*MIN*) e il giorno della settimana (*WEEK_DAY*).

In [82]:
set_df["DATE"] = pd.to_datetime(set_df["DATE"], format="%d/%m/%Y %H:%M", dayfirst=True)
set_df["DAY"] = pd.to_datetime(set_df["DATE"]).dt.day
set_df["MINUTE"] = pd.to_datetime(set_df["DATE"]).dt.hour * 60 + pd.to_datetime(set_df["DATE"]).dt.minute
set_df["WEEK_DAY"] = pd.to_datetime(set_df["DATE"]).dt.day_of_week
del set_df["DATE"]
set_df[:5]

,LINESET,POWER,DAY,MINUTE,WEEK_DAY
0,DG1000420,37.439999,1,0,4
1,DG1000420,37.439999,1,10,4
2,DG1000420,36.000000,1,20,4
3,DG1000420,35.279999,1,30,4
4,DG1000420,35.279999,1,40,4


## Trasformazione corrente in potenza media
Trasformiamo la corrente erogata, in potenza media misurata in KWh.  
Alle ubicazioni civili viene associata una tensione trifase: 240V (RMS).  
Dunque: KWh = kV * I * 1h = 240V/1000 * I * 3600s = 240kV*3.6s * I = 864 KV*s * I  
Definisco quindi la costante di conversione `conv_to_KWh` := 864 KV*s e converto la colonna *POWER* in KWh.


In [86]:
conv_to_KWh = 240/1000*3600
set_df["POWER"] *= conv_to_KWh
set_df[:5]

,LINESET,POWER,DAY,MINUTE,WEEK_DAY
0,DG1000420,32348.159136,1,0,4
1,DG1000420,32348.159136,1,10,4
2,DG1000420,31104.000000,1,20,4
3,DG1000420,30481.919136,1,30,4
4,DG1000420,30481.919136,1,40,4


## Consumo mensile per zona
Quello che vogliamo vedere come prima cosa è qual'è il consumo medio mensile per ogni linea e per ogni zona.  

Quello che faremo sarà: 
* creare un dataframe ** unendo *set_df* e *lin_df* in modo da associare anche ad ogni zona la potenza
    * creiamo dei dataframe temporanei dove associamo 
* creare un dataframe ** con i valori di potenza mediati su tutto il mese
* 


In [144]:
# eseguiamo la media giornaliera della potenza 
mean_df = set_df.groupby("LINESET")["POWER"].sum()/set_df["DAY"].iloc[-1]

# uniamo il dataframe mediato con lin_df preservando le colonne LINESET, POWER, SQUAREID e NR_UBI
mean_df = pd.merge(mean_df,lin_df, on="LINESET", how="right")

# aggiungiamo colonna con il numero totale di ubicazioni per squareid
mean_df["TOT_UBI"] = lin_df.groupby("SQUAREID")["NR_UBI"].transform("sum")

# eseguiamo la media pesata della potenza sul numero di ubicazioni
mean_df["W_POWER"] = mean_df["POWER"] * mean_df["NR_UBI"] / mean_df["TOT_UBI"]
# temp_df = pd.merge(try_df.groupby("SQUAREID")["POWER"].sum(), try_df.groupby("SQUAREID")["NR_UBI"].sum(), on="SQUAREID")
mean_df[1:25]

,LINESET,POWER,SQUAREID,NR_UBI,TOT_UBI,W_POWER
1,DG1031221,5.505828e+06,272,72,72,5.505828e+06
2,DG1031221,5.505828e+06,273,21,21,5.505828e+06
3,DG1003117,7.862083e+06,276,34,34,7.862083e+06
4,DG1003117,7.862083e+06,277,3,3,7.862083e+06
5,DG1031221,5.505828e+06,389,65,65,5.505828e+06
6,DG1031221,5.505828e+06,390,8,8,5.505828e+06
7,DG1003117,7.862083e+06,393,25,25,7.862083e+06
8,DG1031221,5.505828e+06,506,12,12,5.505828e+06
9,DG1031221,5.505828e+06,507,27,27,5.505828e+06
10,DG1031221,5.505828e+06,624,5,5,5.505828e+06


In [125]:
gpd = gp.read_file(file_path_raw + "trentino-grid.geojson")
gpd

DataSourceError: ./data/raw/trentino-grid.geojson: No such file or directory

Divido il data frame *lin_df* in modo da poter contare il numero di ubicazioni presenti per ogni linea elettrica *LINESET* e contare il numero di ubicazioni presenti per ogni casella *SQUAREID*.

Per prima cosa creo un DataFrame temporaneo dove ad ogni linea *LINESET* associo il numero di ubicazioni passanti per essa:

In [69]:
temp_df_line = lin_df.groupby("LINESET")["NR_UBI"].sum().reset_index()
temp_df_line = temp_df_line.rename(columns={"NR_UBI": "UBI_X_LINE"})
temp_df_line[:5]

,LINESET,UBI_X_LINE
0,DG1000011,6
1,DG1000023,62
2,DG1000420,2133
3,DG1000421,1424
4,DG1000422,3423


Poi creo un DataFrame temporaneo dove ad ogni casella *SQUAREID* associo il numero di ubicazioni presenti:

In [70]:
temp_df_square = lin_df.groupby("SQUAREID")["NR_UBI"].sum().reset_index()
temp_df_square = temp_df_square.rename(columns={"NR_UBI": "UBI_X_SQUARE"})
temp_df_square[:5]

,SQUAREID,UBI_X_SQUARE
0,155,4
1,272,72
2,273,21
3,276,34
4,277,3


Dopo aver separato *lin_df* nei due data frame temporanei, li riunisco al data frame *lin_df* e a sua volta li unisco in un data frame finale insieme a *set_df* organizzandoli sotto *LINESET*.

In [71]:
final_df1=pd.merge(lin_df,temp_df_line,on="LINESET")
final_df1[:5]

,SQUAREID,LINESET,NR_UBI,UBI_X_LINE
0,155,DG1031221,4,1380
1,272,DG1031221,72,1380
2,273,DG1031221,21,1380
3,389,DG1031221,65,1380
4,390,DG1031221,8,1380


In [72]:

final_df2=pd.merge(final_df1,temp_df_square,on="SQUAREID")
final_df2[:5]

,SQUAREID,LINESET,NR_UBI,UBI_X_LINE,UBI_X_SQUARE
0,155,DG1031221,4,1380,4
1,272,DG1031221,72,1380,72
2,273,DG1031221,21,1380,21
3,389,DG1031221,65,1380,65
4,390,DG1031221,8,1380,8


In [83]:
final_df = pd.merge(final_df2,set_df, on = "LINESET")
final_df#[:5]

,SQUAREID,LINESET,NR_UBI,UBI_X_LINE,UBI_X_SQUARE,POWER,DAY,MINUTE,WEEK_DAY
0,155,DG1031221,4,1380,4,35.040001,1,0,4
1,155,DG1031221,4,1380,4,30.000000,1,10,4
2,155,DG1031221,4,1380,4,32.399998,1,20,4
3,155,DG1031221,4,1380,4,30.840000,1,30,4
4,155,DG1031221,4,1380,4,30.599998,1,40,4
...,...,...,...,...,...,...,...,...,...
10928875,10984,DG1013504,2,74,2,160.225006,30,1390,5
10928876,10984,DG1013504,2,74,2,161.699997,30,1400,5
10928877,10984,DG1013504,2,74,2,159.600006,30,1410,5
10928878,10984,DG1013504,2,74,2,161.600006,30,1420,5
